In [ ]:
dbutils.widgets.text(name="ckey", defaultValue= " ", label= "Consumer Key")
dbutils.widgets.text(name="csecret", defaultValue= " ", label= "Consumer Secret")
dbutils.widgets.text(name="atoken",defaultValue=" ",label="Access Token")
dbutils.widgets.text(name="asecret", defaultValue= " ", label= "Access Secret")
dbutils.widgets.help()

In [ ]:
import asyncio
from azure.eventhub.aio import EventHubProducerClient
from azure.eventhub import EventData
from tweepy import Stream
from tweepy import OAuthHandler
from tweepy.streaming import StreamListener
import json
import nltk
from textblob import TextBlob, Word, Blobber
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('brown')

In [ ]:
def getblobsentiment(text):
  blob = TextBlob(text)
  
  i=0.00
  for sentence in blob.sentences:
      i=i+sentence.sentiment.polarity
      #print(sentence.sentiment.polarity)
    
  if i >0: 
    return 1
  elif i<0:
    return -1
  else:
    return 0

async def run (text):
  producer = EventHubProducerClient.from_connection_string(conn_str="CONNECTION_STRING",eventhub_name="EVENTHUB_NAME")
  async with producer:
    # Create a batch.
    event_data_batch = await producer.create_batch()
    event_data_batch.add(EventData(text))
    await producer.send_batch(event_data_batch)  

In [ ]:
class listener(StreamListener):
      def on_data(self, data): 
        loop = asyncio.get_event_loop()

        t=json.loads(data)

        d1 = {"text":t["text"] ,"sentiment": getblobsentiment(t["text"]) ,"created_at" :t["created_at"]  }
        print (type(data))     
        loop.run_until_complete(run(json.dumps(d1)))
        print(d1)

        return(True)

      def on_error(self, status):
        print(status)
    
    
ckey=dbutils.widgets.get("ckey")
csecret=dbutils.widgets.get("csecret")
atoken=dbutils.widgets.get("atoken")
asecret=dbutils.widgets.get("asecret")

auth = OAuthHandler(ckey, csecret) 
auth.set_access_token(atoken, asecret)

twitterStream = Stream(auth, listener())
#twitterStream.filter(track=["positive"])
#twitterStream.filter(track=["trump"])

In [ ]:
print(getblobsentiment("I feel good"))